In [1]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

import h3
import pandas as pd
from modules.map_visualization import choropleth_map
from modules.config import H3_RESOLUTION, PATH_TRIPS

In [3]:
trips = pd.read_pickle(PATH_TRIPS)

In [8]:
trips.groupby('vehicleType')['id'].nunique()

vehicleType
bicycle         2989
car             1976
kick_scooter    7802
Name: id, dtype: int64

In [9]:
trips.groupby('provider')['id'].nunique()

provider
bird         833
car2go      1200
circ         934
driveby      201
drivenow     575
fordpass    2248
kvb          741
lime        3509
tier        2526
Name: id, dtype: int64

In [4]:
trips = trips.sample(frac=0.1, random_state=42)

In [5]:
trips['hex_id'] = trips.apply(lambda row : h3.geo_to_h3(row.latitude_start, row.longitude_start, H3_RESOLUTION), axis=1)

In [6]:
trips_grouped = trips.groupby('hex_id').size().to_frame('count').reset_index()

In [7]:
trips_grouped['boundaries'] = trips_grouped['hex_id'].map(lambda hex: {"type": "Polygon", "coordinates": [h3.h3_to_geo_boundary(hex, geo_json=True)]})

In [8]:
dfmap = choropleth_map(trips_grouped, hex_id_field='hex_id', geometry_field='boundaries', value_field='count', layer_name='Choropleth 8', with_legend=True)
dfmap